# Layouts

The `toolbox` provides a framework, which allows defining the layout (structure) of an HDF5 file. In contrast, *conventions* define which attributes are required for all datasets and provide feedback during file creation. *Layouts* define *which* dataset, groups, attributes and properties are expected. Let's learn about it by practical examples:

In [1]:
from h5rdmtoolbox import layout  # import the layout module

Init a layout:

In [2]:
lay = layout.Layout()

A Layout(definition) consists of so-called "layout specifications", which we have to add to the layout object. Currently, there are no specifications:

In [3]:
lay.specifications

[]

## Example data
Let's create some thest data:

In [4]:
import h5rdmtoolbox as h5tbx
with h5tbx.File() as h5:
    h5.create_dataset('u', shape=(3, 5), compression='lzf')
    h5.create_dataset('v', shape=(3, 5), compression='gzip')
    h5.create_group('instruments', attrs={'description': 'Instrument data'})

## 1. Adding a specification

Let's add a specification. For this we call `.add()`. We will add information for a query request, which will be performed later, when we validate a file (layout).

The first argument is the query method. We will use `find` from the database class `h5rdmtoolbox.database.hdfdb.FileDB`. Then we add keyword arguments to be passed to that method.

As a first example, we request the following for all files to be validated with our layout: **all dataset must be compressed with "gzip"**

In [5]:
from h5rdmtoolbox.database import hdfdb

In [6]:
spec_all_dataset = lay.add(
    hdfdb.FileDB.find,  # query function
    flt={},
    objfilter='dataset'
)

spec_compression = spec_all_dataset.add(
    hdfdb.FileDB.find_one,  # query function
    flt={'$compression': 'gzip'}  # query parameter
)

# we added one specification to the layout:
lay.specifications

[LayoutSpecification (kwargs={'flt': {}, 'objfilter': 'dataset'})]

Let's perform the validation. We expect one failed validation, because dataset "u" has the wrong compression:

In [18]:
res = lay.validate(h5.hdf_filename)

2023-12-17_12:28:21,193 ERROR    [core.py:77] Applying spec. "LayoutSpecification (kwargs={'flt': {'$compression': 'gzip'}})" on "<HDF5 dataset "u": shape (3, 5), type "<f4", convention "h5py">" failed. += 1


The error log message shows us that one specification was not successful. The `is_valid()` call therefore will return `False`, too:

In [19]:
res.is_valid()

False

In [20]:
hdfdb.FileDB(h5.hdf_filename).find({'$compression': 'gzip'})

[<LDataset "/v" in "C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\tmp\tmp_1\tmp0.hdf" attrs=()>]

The "compression-specification" got called twice and failed one (for dataset "u"):

In [24]:
spec_compression.n_calls, spec_compression.n_fails

(2, 1)